In [152]:
%reset -f
import warnings
import numpy as np
import os
import json
import pandas as pd
import tweepy
warnings.simplefilter(action='ignore', category=FutureWarning)


In [153]:
# Load Twitter credentials
api_keys = json.load(open('twitter-api-keys.local.json'))
bearer_token = api_keys["bearer_token"]
client = tweepy.Client(
    bearer_token=api_keys['bearer_token'], wait_on_rate_limit=True)

auth = tweepy.OAuthHandler(
    consumer_key=api_keys['api_key'],
    consumer_secret=api_keys['api_key_secret'],
    access_token=api_keys['access_token'],
    access_token_secret=api_keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

me = 'fcx_xm'


In [154]:
# Pandas dataframe to store data
# filename = 'twitter.csv'
# if filename in os.listdir('.'):
#     df = pd.read_csv(filename)
# else:
df = pd.DataFrame(columns=[
    'id',
    'screen_name',
    'followers_count',
    'following_count',
    'verified',
    'followers',
    'friends',
    'mutuals'])


In [155]:
# Get my info and fill in the dataframe
me_info = api.get_user(screen_name=me)
df = df.append({
    'id': me_info.id,
    'screen_name': me_info.screen_name,
    'followers_count': me_info.followers_count,
    'following_count': me_info.friends_count,
    'verified': me_info.verified,
    'followers': me_info.followers_count,
    'friends': me_info.friends_count}, ignore_index=True)


In [156]:
def get_followers(screen_name):
    global df
    ids = []
    for fid in tweepy.Cursor(api.get_follower_ids, screen_name=screen_name, count=5000).items():
        ids.append(fid)

    print('pre-follower-filter:', len(ids))
    ids = list(set(ids) - set(df[df['followers'].notnull()]['id'].tolist()))
    print('post-follower-filter:', len(ids))

    info = []
    for i in range(0, len(ids), 100):
        try:
            chunk = ids[i:i+100]
            info.extend(api.lookup_users(user_id=chunk))
        except:
            import traceback
            traceback.print_exc()
            print('Something went wrong, skipping...')

    followers_list = {}
    for i in info:
        followers_list[i.id] = i.screen_name
        if i.id not in df['id'].values:
            df = df.append({
                'id': i.id, 
                'screen_name': i.screen_name, 
                'followers_count': i.followers_count, 
                'following_count': i.friends_count, 
                'verified': i.verified},
                ignore_index=True)
    
    df.loc[df['screen_name'] == screen_name, 'followers'] = json.dumps(followers_list)
    

In [157]:
def get_friends(screen_name):
    global df
    ids = []
    for fid in tweepy.Cursor(api.get_friend_ids, screen_name=screen_name, count=5000).items():
        ids.append(fid)

    print('pre-friend-filter:', len(ids))
    ids = list(set(ids) - set(df[df['followers'].notnull()]['id'].tolist()))
    print('post-friend-filter:', len(ids))

    info = []
    for i in range(0, len(ids), 100):
        try:
            chunk = ids[i:i+100]
            info.extend(api.lookup_users(user_id=chunk))
        except:
            import traceback
            traceback.print_exc()
            print('Something went wrong, skipping...')

    followers_list = {}
    for i in info:
        followers_list[i.id] = i.screen_name
        if i.id not in df['id'].values:
            df = df.append({
                'id': i.id, 
                'screen_name': i.screen_name, 
                'followers_count': i.followers_count, 
                'following_count': i.friends_count, 
                'verified': i.verified},
                ignore_index=True)
    
    df.loc[df['screen_name'] == screen_name, 'friends'] = json.dumps(followers_list)
    df = df.drop_duplicates(subset=['id'], keep='first')
    


In [168]:
def get_mutuals(screen_name):
    global df
    followers = json.loads(df.loc[df['screen_name'] == screen_name, 'followers'].values[0])
    friends = json.loads(df.loc[df['screen_name'] == screen_name, 'friends'].values[0])
    mutuals = {k: v for k, v in followers.items() if k in friends}
    df.loc[df['screen_name'] == screen_name, 'mutuals'] = json.dumps(mutuals)


In [159]:
get_followers(me)
get_friends(me)
get_mutuals(me)

pre-filter: 114
post-filter: 114


,id,screen_name,followers_count,following_count,verified,followers,friends,mutuals
0,432092673,fcx_xm,114,168,False,"{""1489371551518629888"": ""yoplileila"", ""7540806...","{""1489371551518629888"": ""yoplileila"", ""7549789...","{""1489371551518629888"": ""yoplileila"", ""7540806..."
1,1489371551518629888,yoplileila,530,407,False,NaN,NaN,NaN
2,754080615339876352,PedroMaeb,184,172,False,NaN,NaN,NaN
3,1500108767299620865,aucsi,161,172,False,NaN,NaN,NaN
4,1254865098956574722,nosoyersi,292,256,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
171,19725644,neiltyson,14686040,39,True,NaN,NaN,NaN
172,2267955614,ILLUMlNATlS,989,6,False,NaN,NaN,NaN
173,2174373619,beautifulcarz,1159,7,False,NaN,NaN,NaN
174,18164420,nasahqphoto,2959341,146,True,NaN,NaN,NaN
